In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import seaborn as sns
from datetime import datetime
import gc

data_raw = pd.read_csv("./Data1_raw.csv")
data_demo = data_raw.copy()

# Preprocessing Data1

Dataset link: https://www.kaggle.com/datasets/ananaymital/us-used-cars-dataset?rvi=1

## Part 1: Quick look

In [2]:
data_raw

,vin,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,...,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,ZACNJABB5KPJ92081,35.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,Bayamon,NaN,NaN,...,A,9-Speed Automatic Overdrive,t83804,Latitude FWD,NaN,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,SALCJ2FX1LH858117,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,JF1VA2M67G9829723,35.4 in,NaN,NaN,NaN,Sedan,NaN,Guaynabo,17.0,NaN,...,M,6-Speed Manual,t58994,Base,NaN,AWD,All-Wheel Drive,104.3 in,78.9 in,2016
3,SALRR2RV0L2433391,37.6 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,8-Speed Automatic Overdrive,t86074,V6 HSE AWD,NaN,AWD,All-Wheel Drive,115 in,87.4 in,2020
4,SALCJ2FXXLH862327,38.1 in,NaN,NaN,NaN,SUV / Crossover,NaN,San Juan,NaN,NaN,...,A,9-Speed Automatic Overdrive,t86759,S AWD,NaN,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000035,2GNAXJEV0J6261526,39.7 in,NaN,NaN,NaN,SUV / Crossover,NaN,Fairfield,26.0,NaN,...,A,Automatic,t72936,1.5T LT FWD,NaN,FWD,Front-Wheel Drive,107.3 in,72.6 in,2018
3000036,1GNERFKW0LJ225508,38.4 in,NaN,NaN,NaN,SUV / Crossover,NaN,Vallejo,18.0,NaN,...,A,Automatic,t85763,LS FWD,NaN,FWD,Front-Wheel Drive,120.9 in,78.6 in,2020
3000037,3FA6P0HD3GR134062,38.3 in,NaN,NaN,NaN,Sedan,NaN,Napa,NaN,NaN,...,A,6-Speed Automatic Overdrive,t57569,SE,NaN,FWD,Front-Wheel Drive,112.2 in,83.5 in,2016
3000038,SAJAJ4BNXHA968809,35 in,NaN,NaN,NaN,Sedan,NaN,Fairfield,30.0,NaN,...,A,Automatic,t65977,20d Premium AWD,NaN,AWD,All-Wheel Drive,111.6 in,81.7 in,2017


In [3]:
data_raw.columns

Index(['vin', 'back_legroom', 'bed', 'bed_height', 'bed_length', 'body_type',
       'cabin', 'city', 'city_fuel_economy', 'combine_fuel_economy',
       'daysonmarket', 'dealer_zip', 'description', 'engine_cylinders',
       'engine_displacement', 'engine_type', 'exterior_color', 'fleet',
       'frame_damaged', 'franchise_dealer', 'franchise_make', 'front_legroom',
       'fuel_tank_volume', 'fuel_type', 'has_accidents', 'height',
       'highway_fuel_economy', 'horsepower', 'interior_color', 'isCab',
       'is_certified', 'is_cpo', 'is_new', 'is_oemcpo', 'latitude', 'length',
       'listed_date', 'listing_color', 'listing_id', 'longitude',
       'main_picture_url', 'major_options', 'make_name', 'maximum_seating',
       'mileage', 'model_name', 'owner_count', 'power', 'price', 'salvage',
       'savings_amount', 'seller_rating', 'sp_id', 'sp_name', 'theft_title',
       'torque', 'transmission', 'transmission_display', 'trimId', 'trim_name',
       'vehicle_damage_category', 'whe

Lots of columns, many of which seem to have redundant information and/or lots of NULLs, let's take a closer look.

## Part 2: Dropping useless data

### No valid entries

Some columns only contain null values: `combine_fuel_economy`, `is_certified`, `vehicle_damage_category` -> drop them!

In [4]:
data_raw[["combine_fuel_economy", "is_certified", "vehicle_damage_category"]].isnull().all()

combine_fuel_economy       True
is_certified               True
vehicle_damage_category    True
dtype: bool

### Redundant info

`engine_cylinders` has the exact same entries as `engine_type` -> drop `engine_cylinders`

In [5]:
(data_raw.engine_cylinders.dropna() == data_raw.engine_type.dropna()).all()

True

`exterior_color` is just `listing_color` with more vehicle-specific detail that will bog down modelling -> here we'll just drop `exterior_color`

In [6]:
data_raw.exterior_color.value_counts()

Black                                  162463
White                                  126560
Summit White                            96432
Silver                                  78908
Gray                                    77562
                                        ...  
Blue Dusk Clearcoat Metallic                1
Dark Performancesian Green Metallic         1
BRILLANT                                    1
WHITE MET                                   1
Cosmic                                      1
Name: exterior_color, Length: 28665, dtype: int64

`transmission_display` is just `transmission` with more vehicle-specific detail that will bog down modelling -> here we'll just drop `transmission_display`

`is_cpo` (dealer certified pre-owned) heavily overlaps with `is_oemcpo` (manufacturer certified pre-owned) -> drop `is_cpo`

`wheel_system_display` is just a renaming of `wheel_system` -> drop `wheel_system_display`

### Overly specific/otherwise useless info

The following columns contain either overly specific info (too many categoies) or info that has little to no bearing on the vehicle price:

`back_legroom`, `bed`, `bed_height`, `bed_length`, `cabin`, `description`, `fleet`, `frame_damaged`, `front_legroom`, `interior_color`, `isCab`, `is_new`, `main_picture_url`, `major_options`, `owner_count`, `salvage`, `savings_amount`, `theft_title`

We'll drop them.

### Drop them!

In [7]:
data_demo = data_demo.drop(columns=["back_legroom", "bed", "bed_height", "bed_length",
                                    "cabin", "description", "engine_cylinders",
                                    "combine_fuel_economy", "exterior_color", "fleet", "frame_damaged",
                                    "front_legroom", "interior_color", "isCab", "is_certified",
                                    "is_cpo", "is_new", "main_picture_url", "major_options",
                                    "owner_count", "salvage", "savings_amount", "theft_title",
                                    "transmission_display", "vehicle_damage_category",
                                    "wheel_system_display"])

## Part 3: Dealing with missing values

Even after dropping the most egregious columns when it comes to NULL values, there are still a lot of NULLs in the remaining columns that can mean different things in different contexts. Let's go through them:

In [8]:
data_demo.isnull().sum()

vin                           0
body_type                 13543
city                          0
city_fuel_economy        491285
daysonmarket                  0
dealer_zip                    0
engine_displacement      172386
engine_type              100581
franchise_dealer              0
franchise_make           572635
fuel_tank_volume         159269
fuel_type                 82724
has_accidents           1426595
height                   159269
highway_fuel_economy     491285
horsepower               172386
is_oemcpo               2864678
latitude                      0
length                   159269
listed_date                   0
listing_color                 0
listing_id                    0
longitude                     0
make_name                     0
maximum_seating          159269
mileage                  144387
model_name                    0
power                    481426
price                         0
seller_rating             40872
sp_id                        96
sp_name 

In the case of `has_accidents`, NULL values are actually important and should not be blindly discarded, since not reporting accidents can indeed often mean that minor damage occured. Let's impute a third value "Unreported" for missings in this variable:

In [9]:
data_demo["has_accidents"] = data_demo.has_accidents.fillna("Unreported")

Furthermore, NULLs in `is_oemcpo` (manufacturer certified pre-owned) are also meaningful (actually `is_oemcpo` only has True or NULL entries). The data was scrapped from a number of websites on which a tag will appear if the vehicle was certified and not otherwise: a missing value here is likely to mean that the car was not certified, as opposed to implying a truly unknown value. We can safely fill missings here with "False":

In [10]:
data_demo["is_oemcpo"] = data_demo.is_oemcpo.fillna("False")

Another variable with meaning missing values is `franchise_make`. Indeed, every row that has `franchise_dealer` as False (is not a franchise dealer) would logically have a missing value for what make the franchise was. However, we need to be cautious here, as even for those rows where the dealer was a franchise dealer, the `franchise_make` can still be null:

In [11]:
data_demo[data_demo.franchise_dealer == False].shape

(568970, 40)

In [12]:
data_demo[data_demo.franchise_make.isnull()].shape

(572635, 40)

So we need a slightly less naive imputation strategy for `franchise_make` -> for those rows where `franchise_dealer` is False and `franchise_make` is NULL, we fill the NULL with "Not-Franchise-Dealer", and everywhere else we retain the NULL values:

In [13]:
franchise_make_fillna_mask = (data_demo.franchise_dealer == False)&(data_demo.franchise_make.isnull())
data_demo.loc[franchise_make_fillna_mask, "franchise_make"] = "Not-Franchise-Dealer"

Finally, `power`, `torque`, `city_fuel_economy`, and `highway_fuel_economy` all have a large number of missing values while not being critical variables. Simply dropping them would be to lose a lot of data over what is probably less important information. Therefore, we will fill NULLs in `power` and `torque` with "Unknown" and NULLs in `city_fuel_economy` and `highway_fuel_economy` with "-1":

In [14]:
data_demo["power"] = data_demo.power.fillna("Unknown")
data_demo["torque"] = data_demo.torque.fillna("Unknown")
data_demo["city_fuel_economy"] = data_demo.city_fuel_economy.fillna(-1)
data_demo["highway_fuel_economy"] = data_demo.highway_fuel_economy.fillna(-1)

Now that every meaningful NULL value has been handled, we can drop the rows with unmeaningful, actually missing values:

In [20]:
data_demo = data_demo.dropna()

TODO:
* Everything in `../Data2/Data2_Preprocessing.ipynb`
* Clean `engine_type`

## Final preprocessing function

In [22]:
def preprocess_data1(data):
    # dropping unused columns
    data = data.drop(columns=["back_legroom", "bed", "bed_height", "bed_length",
                              "cabin", "description", "engine_cylinders",
                              "combine_fuel_economy", "exterior_color", "fleet", "frame_damaged",
                              "front_legroom", "interior_color", "isCab", "is_certified",
                              "is_cpo", "is_new", "main_picture_url", "major_options",
                              "owner_count", "salvage", "savings_amount", "theft_title",
                              "transmission_display", "vehicle_damage_category", "wheel_system_display"])
    
    # filling NULLs
    data["has_accidents"] = data.has_accidents.fillna("Unreported")
    data["is_oemcpo"] = data.is_oemcpo.fillna("False")
    franchise_make_fillna_mask = (data.franchise_dealer == False)&(data.franchise_make.isnull())
    data.loc[franchise_make_fillna_mask, "franchise_make"] = "Not-Franchise-Dealer"
    data["power"] = data.power.fillna("Unknown")
    data["torque"] = data.torque.fillna("Unknown")
    data["city_fuel_economy"] = data.city_fuel_economy.fillna(-1)
    data["highway_fuel_economy"] = data.highway_fuel_economy.fillna(-1)
    
    # removing NULLs
    data = data.dropna()
    
    # TODO: more stuff
    
    # return out preprocessed data
    return data

In [23]:
preprocess_data1(data_raw).to_csv("./Data1_partially_preprocessed.csv", na_rep="NULL", index=False)